# One (!) Way of Using Gradients (i.e. Rotations) on a Glacier
#### code by Alexander Wietek and Joachim Wassermann

In [1]:
%matplotlib inline

import os
import matplotlib
matplotlib.use('agg')
from obspy import *
from obspy.clients.filesystem import sds
from obspy.clients.fdsn import Client
import obspy.signal.array_analysis as AA
import numpy as np
import rochade2 as rochade
import pickle
import scipy as sp
from obspy.core import AttribDict
from obspy.signal.rotate import rotate2zne
from pathlib import Path

SDS_DATA_PATH = "/Users/jowa/Skience2026_Data/data_sds/"
INVENTORY = "../stationxml"

In [2]:
client = sds.Client(sds_root = SDS_DATA_PATH)
sstation = 'XG.RAD1..GL*'
rstation = 'XG.RAD1..DJ*'

output_dir = "./Results/Grenzgletscher2024"
Path(output_dir).mkdir(parents=True, exist_ok=True)

ot=start = UTCDateTime("2024-03-20T00:00:00")
endy   = UTCDateTime("2024-03-20T23:00:00")

vmin=100
vmax=2500

starty = start

love_only = False
xcorr = True
rad = False
flinn = True
periods_per_window = 1.
# Specify frequency bands here
f_min = 5.
f_max = 20.
wghtmethod = "normed"
#mvalue=10.e-10
mvalue=-1
exp = 0.3 # exponent of normed weights
df = 50. # common sampling rate to resample the data
body = False
detrigger = False
protrigger = False
tsta = 1.0
tlta = 10.
thres1=2.0
thres2=1.9
pre = 4.
post = 4.


In [3]:
# The lowpass and highpass frequencies for bandpass filtering
f_lower = []
f_higher = []
fcenter= f_max/(np.sqrt(np.sqrt(2.)))
first = True
#we try half octave band filtering
while fcenter > f_min:
   f_l = fcenter/(np.sqrt(np.sqrt(2.)))
   f_u = fcenter*(np.sqrt(np.sqrt(2.)))
   f_lower.append(f_l)
   f_higher.append(f_u)
   fcenter = fcenter/(np.sqrt(2.))

f_lower = np.asarray(f_lower)
f_higher = np.asarray(f_higher)

trigger_params={"sta": tsta,"lta": tlta,"thres_1":thres1,"thres_2": thres2,"pre_t":pre,"post_t":post}
firstRun = True;

end = 0.


In [4]:
while end < endy:
    tsz = []
    tsn = []
    tse = []
    coo = []
    first = True
    if (endy - start) > 3600:
        end = start + 3600.
    else:
        end = endy

    tsz = []
    tsn = []
    tse = []
    coo = []

    net,stat,loc,chan=sstation.split(".")

    # get the relevant information 
    sz1 = client.get_waveforms(network=net,station=stat,location=loc,channel=chan,starttime=start,endtime=end)
    inv = read_inventory("%s/station_%s_%s.xml"%(INVENTORY,net,stat))
    sz1.merge(method=1,fill_value='latest')
    sz1.attach_response(inv)
    sz1.detrend("linear")
    sz1.decimate(4)
    sz1.resample(sampling_rate=df,no_filter=False)
    sz1.sort()
    sz1.reverse()
    sz1.rotate(method='->ZNE', inventory=inv, components=['ZNE'])
    print(sz1)
    end1 = sz1[0].stats.endtime

    sz1.detrend("linear")
    sz1.filter('highpass',freq=0.050,zerophase=True)

    sz1.remove_response(water_level=10, output="VEL")
    sz1.detrend("linear")

    #rotations
    net,stat,loc,chan=rstation.split(".")
    rots = client.get_waveforms(network=net,station=stat,location=loc,channel=chan,starttime=start,endtime=end)
    rots.merge(method=1,fill_value='latest')
    rots.sort()
    rots.reverse()
    print(inv[0][0][0])
    rots.detrend("linear")
    rots.resample(sampling_rate=df,no_filter=False)
    #rots.decimate(4)
    end2 = rots[0].stats.endtime
    print(rots)
    rots.detrend("linear")

    if end1 > end2:
      rots.trim(start+1,end2)
      sz1.trim(start+1,end2)
    else:
      rots.trim(start+1,end1)
      sz1.trim(start+1,end1)
        
    acc = sz1.copy()
    acc.differentiate()
    rot = rots.copy()

###################################################################
# Start of calculations ...
###################################################################

    # and the parameter periods_per_window
#    windows_l = rochade.rolode_estimator( f_lower, f_higher, periods_per_window,vmin,vmax,trigger_params=trigger_params)
#    windows_l.printlog()
    

####################################################################
## Main processing step, window_estimation takes the 
## vertical rotation rate, north acceleration and east acceleration
## and calculates the estimations - LOVE WAVES
####################################################################
  #  if body == True:
#     irot = rot.copy()
#        irot.integrate()
#        windows_l.window_estimation_love(irot[0],sz1[1],sz1[2], verbose = True,mask_value=mvalue,body=body,trigger=detrigger,pro=protrigger)
#        save_path_love = "%s/%s_%02d-%02d%02d-SH/"%(output_dir,stat,starty.day,endy.day,endy.month)

 #   else:
 #       windows_l.window_estimation_love(rot.select(component="Z")[0],acc.select(component="N")[0],acc.select(component="E")[0],
 #                         verbose = True,mask_value=mvalue,trigger=detrigger,pro=protrigger)
 #       save_path_love = "%s/%s_%02d-%02d%02d-Love/"%(output_dir,stat,starty.day,endy.day,endy.month)
 #   if not os.path.exists(save_path_love):
 #      os.makedirs(save_path_love)

 #   windows_l.write(save_path_love + "windows_l.pkl",firstRun)
 #   windows_l.writelog(save_path_love + "info_l.log",firstRun)
##################################################################
# Main processing step, window_estimation takes the 
# vertical acceleration , north and east rotation rate
# and calculates the estimations - RAYLEIGH WAVES
###################################################################
    if love_only == False:
        windows_r2 = rochade.rolode_estimator( f_lower, f_higher, periods_per_window,vmin,vmax,trigger_params=trigger_params)
        windows_r2.printlog()
        if body == True:
            irot = rot.copy()
            irot.integrate()
            save_path_ray2 = "%s/%s_%02d-%02d%02d-SV2/"%(output_dir,stat,starty.day,endy.day,endy.month)
            windows_r2.window_estimation_rayleigh2(sz1,irot.select(component="N")[0],irot.select(component="E")[0],\
                                                   verbose=True,body=body,mask_value=mvalue,trigger=detrigger,pro=protrigger, rad=rad,flinn=flinn)
        else:
            save_path_ray2 = "%s/%s_%02d-%02d%02d-Rayleigh2/"%(output_dir,stat,starty.day,endy.day,endy.month)
            windows_r2.window_estimation_rayleigh2(acc,rot.select(component="N")[0],rot.select(component="E")[0],\
                                                   verbose=True,body=body,mask_value=mvalue,trigger=detrigger,pro=protrigger,rad=rad,flinn=flinn)

        if not os.path.exists(save_path_ray2):
           os.makedirs(save_path_ray2)


        windows_r2.write(save_path_ray2 + "windows_r2.pkl",firstRun)
        windows_r2.writelog(save_path_ray2 + "info_r2.log",firstRun)

    firstRun=False
    start = end
    

3 Trace(s) in Stream:
XG.RAD1..GLZ | 2024-03-20T00:00:00.000000Z - 2024-03-20T00:59:59.980000Z | 50.0 Hz, 180000 samples
XG.RAD1..GLN | 2024-03-20T00:00:00.000000Z - 2024-03-20T00:59:59.980000Z | 50.0 Hz, 180000 samples
XG.RAD1..GLE | 2024-03-20T00:00:00.000000Z - 2024-03-20T00:59:59.980000Z | 50.0 Hz, 180000 samples
Channel 'GLE', Location '' 
	Time range: 2024-03-15T11:00:00.000000Z - 2024-04-14T14:07:00.000000Z
	Latitude: 45.9601, Longitude: 7.8025, Elevation: 2536.6 m, Local Depth: 0.0 m
	Azimuth: 90.00 degrees from north, clockwise
	Dip: 0.00 degrees down from horizontal
	Sampling Rate: 1000.00 Hz
	Sensor (Description): geophone (DTCC; DT-SOLO; Low-Frequency_Corner 5 Hz; Coil_Resistance 1850; Shunt_Resistance 43000; Sensitivity 76.6 V/m/s; Sensor_Type groundVel)
	Response information available
3 Trace(s) in Stream:
XG.RAD1..DJZ | 2024-03-20T00:00:00.000000Z - 2024-03-20T00:59:59.980000Z | 50.0 Hz, 180000 samples
XG.RAD1..DJN | 2024-03-20T00:00:00.000000Z - 2024-03-20T00:59:59.9800

In [5]:
###################################################################
# Postprocessing step
##################################################################

# Load windows from file
#save_path = save_path_love
#winfile = open(save_path + "windows_l.pkl","rb")
#windows_l = pickle.load(winfile)
#print("open %s"%winfile)
#winfile.close()
#windows_l.wineval = True

# Calculate means, modes, medians, standard deviations, and set weights
#windows_l.set_weights(method = wghtmethod, exp = exp)
#windows_l.calc_mean_std()
#windows_l.calc_kde_mode()
#windows_l.calc_median()

# Plot all the histograms for the individual frequencies
#windows_l.plot_histograms(savepath = save_path)

# Plot the slowness and backazimuth distribution for all frequencies
#windows_l.plot_polar(savepath = save_path)

# Plot the dispersion curves with errorbars 
#windows_l.plot_errorbar(savepath = save_path)
# Write results to a file
#windows_l.writeresults(savepath = save_path)

print("\n Done Love")
if love_only == True:
   exit


 Done Love


In [6]:
###################################################################
# Postprocessing step Rayleigh2 
###################################################################
if love_only == False:
    save_path = save_path_ray2
    winfile = open(save_path + "windows_r2.pkl","rb")
    windows_r2 = pickle.load(winfile)
    winfile.close()
    windows_r2.wineval = True

# Calculate means, modes, medians, standard deviations, and set weights
    windows_r2.set_weights(method = wghtmethod, exp = exp)
    windows_r2.calc_mean_std()
    windows_r2.calc_kde_mode()
    windows_r2.calc_median()

# Plot all the histograms for the individual frequencies
    windows_r2.plot_histograms(savepath = save_path)


# Plot the dispersion curves with errorbars 
    windows_r2.plot_errorbar(savepath = save_path)

# Write results to a file
    windows_r2.writeresults(savepath = save_path)

# Plot the slowness and backazimuth distribution for all frequencies
    windows_r2.plot_polar(savepath = save_path)
    print("\n Done Rayleigh2")

print("\n DONE ALL")


Calculating means and standard deviations...

  Now at frequency f = 16.82
  Now at frequency f = 11.89
  Now at frequency f = 8.41
  Now at frequency f = 5.95
Calculating KDEs and modes:

  Now at frequency f = 17.07
  Now at frequency f = 12.07


/Users/jowa/MESS2026_Jo/6C-Gradients/rochade2.py:700: UserWarning: Not able to calculate kde ... continuing
  warnings.warn(msg)
/Users/jowa/MESS2026_Jo/6C-Gradients/rochade2.py:532: UserWarning: Not able to calculate median.... continuing
  warnings.warn(msg)


  Now at frequency f = 8.54
  Now at frequency f = 6.04

Calculating medians...

  Now at frequency f = 17.07
  Now at frequency f = 12.07
  Now at frequency f = 8.54
  Now at frequency f = 6.04
Plotting histograms ...
Plotting Dispersion curve with errorbars


/Users/jowa/MESS2026_Jo/6C-Gradients/rochade2.py:1016: UserWarning: Nothing to plot...
  warnings.warn(msg)



 Done Rayleigh2

 DONE ALL


/Users/jowa/MESS2026_Jo/6C-Gradients/rochade2.py:825: UserWarning: Nothing to plot ... continuing
  warnings.warn(msg)
